In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
pwd

'/content/gdrive/My Drive/Kaggle'

In [ ]:
!kaggle datasets download -d puneet6060/intel-image-classification

intel-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import tensorflow as tf
import numpy as np 

from zipfile import ZipFile


In [ ]:
!pip install ipython-autotime

%load_ext autotime

time: 131 µs (started: 2021-04-29 19:14:25 +00:00)


In [ ]:
strategy = tf.distribute.MirroredStrategy()
print(strategy.num_replicas_in_sync)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
1
time: 5.14 s (started: 2021-04-29 19:14:25 +00:00)


In [ ]:
import os 
import pathlib 

files = os.path.join(os.getcwd(), 'temp/seg_train/seg_train')

files = pathlib.Path(files)

time: 2.62 ms (started: 2021-04-29 19:14:30 +00:00)


In [ ]:
test_files = os.path.join(os.getcwd(), 'temp/seg_test/seg_test')
test_files  = pathlib.Path(test_files)

time: 1.25 ms (started: 2021-04-29 19:14:30 +00:00)


In [ ]:
filenames = list(files.glob('*/*.jpg'))
testnames = list(test_files.glob('*/*.jpg'))
print(len(filenames))
print(len(testnames))

9418
2499
time: 26.6 s (started: 2021-04-29 19:14:30 +00:00)


In [ ]:
fnames = []
for fname in filenames: 
  fnames.append(str(fname))
np.random.shuffle(fnames)


time: 15.9 ms (started: 2021-04-29 19:14:56 +00:00)


In [ ]:
tnames = []
for tname in testnames: 
  tnames.append(str(tname))
np.random.shuffle(tnames)

time: 6.12 ms (started: 2021-04-29 19:14:56 +00:00)


In [ ]:
class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea']

time: 2.32 ms (started: 2021-04-29 19:14:57 +00:00)


In [ ]:
file_ds = tf.data.Dataset.from_tensor_slices(fnames)
ds_size = file_ds.cardinality().numpy()
print(ds_size)

9418
time: 49.3 ms (started: 2021-04-29 19:14:57 +00:00)


In [ ]:
test_file_ds = tf.data.Dataset.from_tensor_slices(tnames)

time: 9.83 ms (started: 2021-04-29 19:14:57 +00:00)


In [ ]:
batch_size = 128
replica = strategy.num_replicas_in_sync
global_batch_size = batch_size * replica 

time: 851 µs (started: 2021-04-29 19:14:57 +00:00)


In [ ]:
 from IPython.core.debugger import set_trace

time: 720 µs (started: 2021-04-29 19:14:57 +00:00)


In [ ]:

def getLabel(filePath):
  parts = tf.strings.split(filePath, '/')
  fileName = parts[-2]
  labels = fileName == class_names
  labels = tf.cast(labels, tf.int64)
  return labels 

time: 4.04 ms (started: 2021-04-29 19:14:57 +00:00)


In [ ]:

def getImg(filePath):
  img = tf.io.read_file(filePath)
  img = tf.image.decode_jpeg(img, 3)
  img = tf.image.resize(img,[180,180])
  img = img/255.0
  return img 


time: 4.6 ms (started: 2021-04-29 19:14:57 +00:00)


In [ ]:
@tf.function 
def normalize(filePath):
  
  labels = getLabel(filePath)
  img = getImg(filePath)
  return img, labels

time: 1.45 ms (started: 2021-04-29 19:14:57 +00:00)


In [ ]:
def performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(ds_size)
  ds = ds.batch(global_batch_size)
  ds = ds.prefetch(AUTOTUNE)
  return ds

time: 2.92 ms (started: 2021-04-29 19:14:57 +00:00)


In [ ]:
import tensorflow as tf
from tensorflow.data import AUTOTUNE 

train_ds = file_ds.map(normalize, num_parallel_calls=AUTOTUNE, deterministic=False)
test_ds = test_file_ds.map(normalize, num_parallel_calls=AUTOTUNE, deterministic=False)




time: 198 ms (started: 2021-04-29 19:14:57 +00:00)


In [ ]:
train_ds = train_ds.cache()
train_ds = train_ds.batch(global_batch_size)
train_ds = train_ds.prefetch(AUTOTUNE)

time: 5.7 ms (started: 2021-04-29 19:14:57 +00:00)


In [ ]:
test_ds = test_ds.cache().batch(global_batch_size).prefetch(AUTOTUNE)

time: 3.31 ms (started: 2021-04-29 19:14:57 +00:00)


In [ ]:
for img, label in test_ds.take(1):
  print(label)

In [ ]:

train_dist_ds = strategy.experimental_distribute_dataset(train_ds)
test_dist_ds = strategy.experimental_distribute_dataset(test_ds)

time: 32.1 ms (started: 2021-04-29 19:15:29 +00:00)


In [ ]:
from tensorflow.keras import Model, layers, Sequential

def createModel():
  model = Sequential(
      [
        layers.Conv2D(32, 3, activation='relu'),
                      layers.MaxPooling2D(),
                      layers.Conv2D(64, 3, activation ='relu'),
                      layers.MaxPooling2D(),
                      layers.Flatten(),
                      layers.Dense(64, activation='relu'),
                      layers.Dense(5, activation = 'softmax')
      ]
  )
  return model 

time: 4.38 ms (started: 2021-04-29 19:15:29 +00:00)


In [ ]:
with strategy.scope():
  loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits = False, 
                                                               reduction = tf.keras.losses.Reduction.NONE)
  def compute_loss(labels, predictions):
    per_example_loss = loss_object(labels, predictions)
    #print('per_example_loss', per_example_loss)
    return tf.nn.compute_average_loss(per_example_loss,global_batch_size = global_batch_size)

  train_accuracy = tf.keras.metrics.CategoricalAccuracy()
  test_accuracy = tf.keras.metrics.CategoricalAccuracy()
  optimizer = tf.keras.optimizers.Adam()
  model = createModel()


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [ ]:
@tf.function
def distributed_train(inputs):
    per_replica_loss = strategy.run(train_step, args = (inputs,))

    foo = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_loss, axis = None)
    return foo 
  
def train_step(inputs):

  images, labels = inputs

  with tf.GradientTape() as tape:
    predictions = model(images, training = True)
    loss = compute_loss(labels, predictions)
  
  gradients = tape.gradient(loss, model.trainable_weights)
  # set_trace()
  optimizer.apply_gradients(zip(gradients, model.trainable_weights))
  train_accuracy.update_state(labels, predictions)
  return loss 

time: 7.46 ms (started: 2021-04-29 19:15:29 +00:00)


In [ ]:
@tf.function 
def distributed_test(input):
  return strategy.run(test_step, args = (input,))  

def test_step(input):
  images, labels = input 
  preds = model(images)
  t_loss = compute_loss(labels, preds)

  test_accuracy.update_state(labels, preds)

time: 4.29 ms (started: 2021-04-29 19:38:16 +00:00)


In [ ]:
epochs = 40 

for epoch in range(epochs):
  total_loss = 0.0
  num_batches = 0

  for x in train_dist_ds:
    total_loss += distributed_train(x)

    num_batches += 1
  for batch in test_dist_ds:
    distributed_test(batch)

  train_loss = total_loss/num_batches 
  acc = train_accuracy.result()
  test_acc = test_accuracy.result()
  print('epoch', epoch,  'loss', train_loss, 'acc', acc, 'test_acc', test_acc)
  train_accuracy.result()
  test_accuracy.result()


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
epoch 0 loss tf.Tensor(0.36629972, shape=(), dtype=float32) acc tf.Tensor(0.7900828, shape=(), dtype=float32) test_acc tf.Tensor(0.6802721, shape=(), dtype=float32)
epoch 1 loss tf.Tensor(0.2859738, shape=(), dtype=float32) acc tf.Tensor(0.8188575, shape=(), dtype=float32) test_acc tf.Tensor(0.6652661, shape=(), dtype=float32)
epoch 2 loss tf.Tensor(0.23547769, shape=(), dtype=float32) acc tf.Tensor(0.8387768, shape=(), dtype=float32) test_acc tf.Tensor(0.6700013, shape=(), dtype=float32)
epoch 3 loss tf.Tensor(0.18617956, shape=(), dtype=float32) acc tf.Tensor(0.8555072, shape=(), dtype=float32) test_acc tf.Tensor(0.66236496, shape=(), dtype=float32)
epoch 4 loss tf.Tensor(0.14192279, shape=(), dtype=float32) 

In [ ]:
for x in train_dist_ds:
  img, label = x 
  print(tf.shape(img))
  print(tf.shape(label))